# Import and install dependencies


In [1]:
# !pip install tensorflow tensorflow-gpu opencv-python mediapipe sklearn matplotlib

In [2]:
import cv2
import numpy as np
import os
from matplotlib import pyplot as plt
import time
import mediapipe as mp

objc[63424]: Class CaptureDelegate is implemented in both /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x137476480) and /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_videoio.3.4.16.dylib (0x13dfc8860). One of the two will be used. Which one is undefined.
objc[63424]: Class CVWindow is implemented in both /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1374764d0) and /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x117794a68). One of the two will be used. Which one is undefined.
objc[63424]: Class CVView is implemented in both /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/cv2/cv2.abi3.so (0x1374764f8) and /Users/damura/opt/miniconda3/envs/tensorflow/lib/python3.9/site-packages/mediapipe/.dylibs/libopencv_highgui.3.4.16.dylib (0x117794a90). One 

# Extract landmarks using MediaPipe Holistic

Get MediaPipe Holistic model to use it.

In [3]:
mp_holistic = mp.solutions.holistic

Get MediaPipe drawing utilities for the landmarks.

In [4]:
mp_drawing = mp.solutions.drawing_utils

We define a function that, given an image and a MediaPipe model, is going to process the image and detect all available landmarks. 

Check the following link to discover all the MediaPipe models: https://google.github.io/mediapipe/solutions/solutions.html

For a landmark to be detected, it must appear on the image. For example, if in the image only appears from our waist up, it will never detect the landmark of our feet.

By default, OpenCV reads image on the channel format of BGR. In order to use MediaPipe, we need to convert it to RGB channel format.

In [5]:
def mediapipe_detection(image, model):

  # Converting color from BGR to RGB
  image = cv2.cvtColor(image, cv2.COLOR_BGR2RGB)

  # To improve performance, set image as no longer writeable
  image.flags.writeable = False

  # Get detection results
  results = model.process(image)

  # Set image writeable again
  image.flags.writeable = True

  # Convert color to original (from RGB to BGR)
  image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)

  return image, results


We define a function that will be responsible of drawing all the detected landmarks on an image.

In [6]:
def draw_landmarks(image, results):
            
    # Draw face connections
    mp_drawing.draw_landmarks(image, results.face_landmarks, mp_holistic.FACEMESH_TESSELATION,
                             mp_drawing.DrawingSpec(color=(255,87,51), thickness=1, circle_radius=1),
                             mp_drawing.DrawingSpec(color=(77,255,51), thickness=1, circle_radius=1)
                             )
    
    # Draw pose connections
    mp_drawing.draw_landmarks(image, results.pose_landmarks, mp_holistic.POSE_CONNECTIONS,
                             mp_drawing.DrawingSpec(color=(255,87,51), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(77,255,51), thickness=2, circle_radius=2)
                             ) 
                              
    # Draw left hand connections
    mp_drawing.draw_landmarks(image, results.left_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(255,87,51), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(77,255,51), thickness=2, circle_radius=2)
                             ) 
                              
    # Draw right hand connections  
    mp_drawing.draw_landmarks(image, results.right_hand_landmarks, mp_holistic.HAND_CONNECTIONS, 
                             mp_drawing.DrawingSpec(color=(255,87,51), thickness=2, circle_radius=4), 
                             mp_drawing.DrawingSpec(color=(77,255,51), thickness=2, circle_radius=2)
                             ) 

Set up minimum detection confidence for the model.

In [7]:
min_detection = 0.9

Set up minimum tracking confidence for the model.

In [8]:
min_tracking = 0.9

In order to test the MediaPipe model and extract landmarks, we are going to use the webcam to capture video.

In [9]:
camera = cv2.VideoCapture(0)

Set up MediaPipe Holistic model and start to detect for each frame.

In [10]:
with mp_holistic.Holistic(min_detection_confidence=min_detection, min_tracking_confidence=min_tracking) as holistic:
  while camera.isOpened():

    # Read frame from the camera
    ret, frame = camera.read()

    # Make detections using MediaPipe Holistic
    image, results = mediapipe_detection(frame, holistic)

    # Draw landmarks
    draw_landmarks(image, results)
                   
    # Show frame with landmarks to screen on a window
    cv2.imshow("Real-time detection",image)

    # Check if key 'q' has been pressed to stop recording
    if cv2.waitKey(10) & 0xFF == ord('q'):
      break
    
  # Stop using camera
  camera.release()

  # Destroy window
  cv2.destroyAllWindows()

INFO: Created TensorFlow Lite XNNPACK delegate for CPU.
